# FastAPI Authentication with Google OAuth - Part 1
> Using Google OAuth as your authentication layer in FastAPI.

- toc: true
- badges: true
- comments: true
- categories: [fastapi, google, oauth]
- hide: false

## background

tbd

## Google OAuth Credentials

TODO; .env file

## FastAPI

### Imports

At the top of our Python file, we have our imports.

In [1]:
from typing import Optional
from fastapi import FastAPI, Depends, HTTPException
from fastapi.openapi.docs import get_swagger_ui_html
from fastapi.openapi.utils import get_openapi

from starlette.config import Config
from starlette.requests import Request
from starlette.middleware.sessions import SessionMiddleware
from starlette.responses import HTMLResponse, JSONResponse, RedirectResponse

from authlib.integrations.starlette_client import OAuth

The first `Optional` and `FastAPI` imports are standard among FastAPI applications. I want to explicitly call out `Depends` and `HTTPException`, which will be used to validate a user's login credentials prior to an API call.

The `Starlette` and `AuthLib` imports are the key to our Google OAuth implementation. They'll handle all of the heavy lifting, specifically the redirect URI generation and callback handling. Props to their developers as they make our implementation a breeze!

### FastAPI Setup

Here we initialize our `FastAPI` instance with a root homepage. If the user is not logged in, it will display a login link. If the user is logged in, it will display their email address as well as links to the documentation and logout.

In [3]:
app = FastAPI(docs_url=None, redoc_url=None)
app.add_middleware(SessionMiddleware, secret_key='!secret')

@app.get('/')
async def home(request: Request):
    user = request.session.get('user')
    if user is not None:
        email = user['email']
        html = (
            f'<pre>Email: {email}</pre><br>'
            '<a href="/docs">documentation</a><br>'
            '<a href="/logout">logout</a>'
        )
        return HTMLResponse(html)
    return HTMLResponse('<a href="/login">login</a>')

Rather than saving our authentication to a cookie manually, I've opted to use Starlettes's `SessionMiddleware`. FastAPI has good [documentation](https://fastapi.tiangolo.com/tutorial/middleware/) on middleware, and the key thing to note is that it can perform operations on requests prior to being passed through to the rest of the application as well before delivering responses. Starlette's `SessionMiddleware` [documentation](https://www.starlette.io/middleware/#sessionmiddleware) explains how it utilizes this technique to add a signed cookie to the user session that is readable but not modifiable. Still confused about what middleware is? Check out the diagram below.

<center><img src="https://miro.medium.com/max/1358/1*4nJJgPOnlJwD6s-7ygqgTg.jpeg" width=70%></center>
Image source: <a href="https://medium.com/@tony_starkk/laravel-5-8-middleware-tutorial-with-example-2019-9f5069b8c3cc">link</a>

### Google OAuth Implementation

### Protected Docs

## Credits

This tutorial is based on work originally written by Nils de Bruin on [Medium](https://medium.com/data-rebels/fastapi-google-as-an-external-authentication-provider-3a527672cf33).